<a href="https://colab.research.google.com/github/ChristeenTJose/Signature-similarity-search/blob/main/Signature_similarity_search_using_tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import numpy as np

In [ ]:
os.chdir('/content/drive/MyDrive/Signature similarity search')

original_model = tf.keras.models.load_model('Model256.keras')
original_model.trainable = False
original_model.summary()

print()

embedding_model = tf.keras.models.Sequential(
    original_model.layers[:-1]
)
embedding_model.trainable = False
embedding_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_128 (Functional) │ (None, 4, 4, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 256)            │     4,195,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,620,036 (44.33 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,424,705 (28.32 MB)

...

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_128 (Functional) │ (None, 4, 4, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 256)            │     4,195,584 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,424,448 (28.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,424,448 (28.32 MB)

In [ ]:
os.chdir('/content/drive/MyDrive/Signature similarity search/samples')

reference_image = '1_G1.png'
query_images = ("1_G1.png", "1_G2.png", "1_F2.png")

img = tf.keras.preprocessing.image.load_img(reference_image, target_size=(128, 128))

img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0

img_array = np.expand_dims(img_array, axis=0)

reference_embedding = embedding_model.predict(img_array)[0]

y = tf.convert_to_tensor(reference_embedding, dtype=tf.float32)

for query_image_name in query_images:
  print(f"Querying image: {query_image_name}")
  img = tf.keras.preprocessing.image.load_img(query_image_name, target_size=(128, 128))

  img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0

  img_array = np.expand_dims(img_array, axis=0)

  x = embedding_model.predict(img_array)[0]

  #distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(y, x))))

  distance = tf.reduce_sum(tf.square(tf.subtract(y, x)))

  print(f"Distance: {distance}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Querying image: 1_G1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Distance: 0.0
Querying image: 1_G2.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Distance: 66.81405639648438
Querying image: 1_F2.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Distance: 56.58610534667969


In [ ]:
'''
  When x is 2D
  x - y uses broadcasting to subtract the 1D tensor y from each row of the 2D tensor x.

  tf.square(...) applies element-wise squaring to the result.

  tf.reduce_sum(..., axis=1) sums across the feature dimension d, producing a scalar distance for each row in x.
'''